In [3]:
import sys
sys.path.append('..')

In [4]:
from src.database.database_extract import request_local_api
extract = request_local_api()

#### Extracting examples based on rowID

In [5]:
example = extract.getExample(rowId=1)
example

AttributeError: 'request_local_api' object has no attribute 'cursor'

#### Extracting all the examples in the DemoDatabase

In [6]:
example_all = extract.getAllExamples()
example_all

[(1,
  '0-based interbase',
  'DNA',
  'C',
  'A',
  12345,
  12346,
  '3',
  'GRCh38',
  'NC_000001.10',
  'heterozygous',
  'GENE1',
  123,
  'NC_000001.10:12345:C:A',
  'spdi_str',
  'NC_000001.10:12345:C:A',
  'NC_000001.10:g.12346C>A',
  {'_id': 'ga4gh:VA.CWDQtAfFv3QnUqN3up3J1bs32JVSKFPk',
   'type': 'Allele',
   'state': {'type': 'LiteralSequenceExpression', 'sequence': 'A'},
   'location': {'_id': 'ga4gh:VSL.ixj_LC0E_4GQvm7dtG4r5gjf88lWRxU1',
    'type': 'SequenceLocation',
    'interval': {'end': {'type': 'Number', 'value': 12346},
     'type': 'SequenceInterval',
     'start': {'type': 'Number', 'value': 12345}},
    'sequence_id': 'ga4gh:SQ.S_KjnFVz-FE7M0W6yoaUDgYxLPc1jyWU'}}),
 (2,
  '0-based interbase',
  'DNA',
  'A',
  'ACAG',
  55972968,
  55972972,
  '4',
  'GRCh38',
  'NC_000004.11',
  'homozygous',
  'KDR',
  1234,
  'NC_000004.11:55972968:A:ACAG',
  'spdi_str',
  'NC_000004.11:55972968:A:ACAG',
  'NC_000004.11:g.55972971_55972973dup',
  {'_id': 'ga4gh:VA.niXsdxsUqpwZ

#### Converting a row from the data from DemoCoreLiteDB into a core variant class

In [7]:
example_cvc = extract.db_to_cvc(rowId=1)
example_cvc

{'origCoordSystem': '0-based interbase',
 'seqType': 'DNA',
 'allelicState': 'heterozygous',
 'geneSymbol': 'GENE1',
 'hgncId': 123,
 'refAllele': 'C',
 'altAllele': 'A',
 'chrom': '3',
 'genomeBuild': 'GRCh38',
 'start': 12345,
 'end': 12346,
 'sequenceId': 'NC_000001.10'}